In [ ]:
# J. Ryu, Electron Microscopy and Spectroscopy Lab., Seoul National University
from scipy import stats
from scipy.signal import periodogram
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.widgets import RectangleSelector
import tkinter.filedialog as tkf
import hyperspy.api as hys
from scipy import optimize
import ipywidgets as pyw
import time
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from sklearn.cluster import DBSCAN

In [ ]:
# refer to https://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape) # row, col
    x = (X*data).sum()/total # row
    y = (Y*data).sum()/total # col
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum()) # row
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum()) # col
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [ ]:
# refer to "github.com/mkolopanis/python/blob/master/radialProfile.py"

def radial_stats(image, center=None, var=True):
   
    y, x = np.indices(image.shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    r = np.hypot(y - center[0], x - center[1])
    #plt.imshow(r, cmap="Accent")
    #plt.show()

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = np.around(r_sorted)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    #print(nr)
    
    csim = np.cumsum(i_sorted, dtype=float)
    sq_csim = np.cumsum(np.square(i_sorted), dtype=float)
    radial_avg  = (csim[rind[1:]] - csim[rind[:-1]]) / nr
    
    if var:    
        avg_square = np.square(radial_avg)
        square_avg = (sq_csim[rind[1:]] - sq_csim[rind[:-1]]) / nr
        mask = avg_square.copy()
        mask[np.where(avg_square==0)] = 1.0
        radial_var = (square_avg - avg_square) / mask
        return r, radial_avg, radial_var, (square_avg - avg_square)
    
    else:
        return r, radial_avg

In [ ]:
def gaussian_center(image, cbox_edge=0):
    y, x = np.indices(image.shape)
    if not cbox_edge:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    else:
        cbox_outy = int(image.shape[0]/2 - cbox_edge/2)
        cbox_outx = int(image.shape[1]/2 - cbox_edge/2)
        center_box = image[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
        fit_params = fitgaussian(center_box)
        (_, center_y, center_x, _, _) = fit_params
        center = [center_y+cbox_outy, center_x+cbox_outx]
        
    return center

In [ ]:
def remove_center_beam(image, center=None, cb_rad=0):
    y, x = np.indices(image.shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    r = np.hypot(y - center[0], x - center[1])
    r = np.around(r)
    ri = np.where(r<=cb_rad)
    #print(ri[0].shape)
    
    image[ri] = 0
    
    return image

In [ ]:
def load_binary_4D_stack(img_adr, datatype, original_shape, final_shape, log_scale=False):
    stack = np.fromfile(img_adr, dtype=datatype)
    stack = stack.reshape(original_shape)
    print(stack.shape)
    if log_scale:
        stack = np.log(stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]])
    else:
        stack = stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]]
    
    print(stack.shape) 
    return stack

In [ ]:
def fourd_roll_axis(stack):
    stack = np.rollaxis(np.rollaxis(stack, 2, 0), 3, 1)
    return stack

# load & prepare

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
datatype = "float32"
#o_shape = (128, 128, 130, 128)
#f_shape = (128, 128, 128, 128)
o_shape = (256, 256, 130, 128)
f_shape = (256, 256, 128, 128)

# load a data (RAW)
stack_4d = load_binary_4D_stack(raw_adr, datatype, o_shape, f_shape, log_scale=False)
if np.isnan(stack_4d):
    stack_4d = np.nan_to_num(stack_4d)
print(np.max(stack_4d))
print(np.min(stack_4d))
print(np.mean(stack_4d))

In [ ]:
# Load a data (DM)
stack_4d = hys.load(raw_adr).data
print(stack_4d.shape)
stack_4d = fourd_roll_axis(stack_4d)
f_shape = stack_4d.shape
print(f_shape)

stack_4d = stack_4d / np.max(stack_4d)
print(np.max(stack_4d))
print(np.min(stack_4d))
print(np.mean(stack_4d))

In [ ]:
# rotate
stack_4d = np.rot90(stack_4d, 2, (0, 1))
f_shape = stack_4d.shape
print(f_shape.shape)

In [ ]:
# select an interesting area
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.sum(stack_4d, axis=(2, 3)), cmap="gray")

def onselect(eclick, erelease):
    print('startposition: (%f, %f)' % (eclick.xdata, eclick.ydata))
    print('endposition  : (%f, %f)' % (erelease.xdata, erelease.ydata))

box = RectangleSelector(ax, onselect)
plt.show()

In [ ]:
# crop the data
stack_4d_cropped = stack_4d[int(box.corners[1][0]):int(box.corners[1][2]), 
                                           int(box.corners[0][0]):int(box.corners[0][1])].copy()
cr_shape = stack_4d_cropped.shape
print(cr_shape)
print(np.max(stack_4d_cropped))
print(np.min(stack_4d_cropped))
print(np.mean(stack_4d_cropped))

In [ ]:
# just copy
stack_4d_cropped = stack_4d.copy()
cr_shape = stack_4d_cropped.shape
print(cr_shape)
print(np.max(stack_4d_cropped))
print(np.min(stack_4d_cropped))
print(np.mean(stack_4d_cropped))

In [ ]:
%matplotlib inline

In [ ]:
# maximum intensity distribution
max_int = np.max(stack_4d_cropped, axis=(2, 3))
print(max_int.shape)
print(np.max(max_int))
print(np.min(max_int))
print(np.mean(max_int))
print(np.median(max_int))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(max_int, cmap="afmhot")
ax[0].axis("off")
ax[1].hist(max_int.flatten(), bins=len(max_int))
ax[1].grid()
plt.show()

In [ ]:
# total intensity distribution
tot_int = np.sum(stack_4d_cropped, axis=(2, 3))
print(max_int.shape)
print(np.max(tot_int))
print(np.min(tot_int))
print(np.mean(tot_int))
print(np.median(tot_int))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(tot_int, cmap="afmhot")
ax[0].axis("off")
ax[1].hist(tot_int.flatten(), bins=len(max_int))
ax[1].grid()
plt.show()

# center management

In [ ]:
%matplotlib inline

In [ ]:
# find center position
center_pos = []
for i in range(cr_shape[0]):
    for j in range(cr_shape[1]):
        center_pos.append(gaussian_center(stack_4d_cropped[i, j], cbox_edge=20))
        
center_pos = np.asarray(center_pos)
center_pos = np.reshape(center_pos, (cr_shape[0], cr_shape[1], -1))
print(center_pos.shape)
center_mean = np.mean(center_pos, axis=(0, 1))
print(center_mean)

# center distibution
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].hist(center_pos[:, :, 0].flatten(), bins=100, density=True, color="orange", label="center y position")
ax[0].hist(center_pos[:, :, 1].flatten(), bins=100, density=True, color="gray", alpha=0.5, label="center x position")
ax[0].grid()
ax[0].legend()

ax[1].scatter(center_pos[:, :, 1], center_pos[:, :, 0], s=10.0, alpha=0.5)
ax[1].grid()
ax[1].scatter(center_mean[1], center_mean[0], s=20, c="red")
ax[1].set_xlabel("center x position", fontsize=10)
ax[1].set_ylabel("center y position", fontsize=10)
fig.tight_layout()
plt.show()

In [ ]:
# remove center beam
center_radius = 10
for i in range(cr_shape[0]):
    for j in range(cr_shape[1]):
        stack_4d_cropped[i, j] = remove_center_beam(stack_4d_cropped[i, j], 
                                                    center=center_mean.tolist(), cb_rad=center_radius)
print(np.max(stack_4d_cropped))
print(np.min(stack_4d_cropped))
print(np.mean(stack_4d_cropped))

# rotational average

In [ ]:
# radial average of DPs (not variance, intensity direcltly, RDF?)
radial_avg_stack = []
len_profile = []
for i in range(cr_shape[0]):
    for j in range(cr_shape[1]):
        _, radial_temp = radial_stats(stack_4d_cropped[i, j], center=center_mean.tolist(), var=False)
        len_profile.append(len(radial_temp))
        radial_avg_stack.append(radial_temp)

if len(np.unique(len_profile)) > 1:
    print(np.unique(len_profile))
    shortest = np.min(len_profile)
    for i in range(len(radial_avg_stack)):
        radial_avg_stack[i] = radial_avg_stack[i][:shortest]

radial_avg_stack = np.asarray(radial_avg_stack).reshape(cr_shape[0], cr_shape[1], -1)
print(radial_avg_stack.shape)

radial_avg_sum = np.sum(radial_avg_stack, axis=(0, 1))
print(radial_avg_sum.shape)

plt.figure(figsize=(10,10))
plt.plot(radial_avg_sum)
plt.grid()
plt.show()

In [ ]:
# save (radial average, 3D)
radial_avg_stack = hys.signals.Signal1D(radial_avg_stack)
radial_avg_stack.metadata.General.set_item("notes", center_mean.tolist())
radial_avg_stack.save(raw_adr[:-4]+"_radial_avg_radial.hdf5")

# FEM (variance)

In [ ]:
%matplotlib qt

In [ ]:
%matplotlib inline

In [ ]:
# obtain variance map dpending on k-vector
square_avg = np.mean(np.square(stack_4d_cropped), axis=(0,1))
avg_square = np.square(np.mean(stack_4d_cropped, axis=(0,1)))
mask = avg_square.copy()
mask[np.where(avg_square == 0)] = 1.0
var_map = (square_avg - avg_square) / mask
var_nonor = square_avg - avg_square

# obtain radial average prifile for variance map above
radial_map, kp_var = radial_stats(var_map, center=center_mean.tolist(), var=False)
print(kp_var.shape)
print(np.argmax(kp_var))
_, kp_nonor = radial_stats(var_nonor, center=center_mean.tolist(), var=False)
print(kp_nonor.shape)
print(np.argmax(kp_nonor))

fig, ax = plt.subplots(1, 5, figsize=(15, 3))
ax[0].imshow(radial_map, cmap="gray")
ax[0].axis("off")
ax[1].imshow(var_map, cmap="afmhot")
ax[1].axis("off")
ax[2].plot(kp_var, "k-")
ax[2].grid()
ax[3].imshow(var_nonor, cmap="afmhot")
ax[3].axis("off")
ax[4].plot(kp_nonor, "k-")
ax[4].grid()
plt.show()

In [ ]:
# calculate variance with all angles at a certain k
radial_var_stack = []
radial_var_nonor_stack = []
len_profile = []
for i in range(cr_shape[0]):
    for j in range(cr_shape[1]):
        _, _, radial_temp, radial_nonor = radial_stats(stack_4d_cropped[i, j], center=center_mean.tolist(), var=True)
        len_profile.append(len(radial_temp))
        radial_var_stack.append(radial_temp)
        radial_var_nonor_stack.append(radial_nonor)

if len(np.unique(len_profile)) > 1:
    print(np.unique(len_profile))
    shortest = np.min(len_profile)
    for i in range(len(len_profile)):
        radial_var_stack[i] = radial_var_stack[i][:shortest]
        radial_var_nonor_stack[i] = radial_var_nonor_stack[i][:shortest]

radial_var_stack = np.asarray(radial_var_stack).reshape(stack_4d_cropped.shape[0], stack_4d_cropped.shape[1], -1)
print(radial_var_stack.shape)

radial_var_sum = np.sum(radial_var_stack, axis=(0, 1))
print(radial_var_sum.shape)

radial_var_nonor_stack = np.asarray(radial_var_nonor_stack).reshape(stack_4d_cropped.shape[0], stack_4d_cropped.shape[1], -1)
print(radial_var_nonor_stack.shape)

radial_var_nonor_sum = np.sum(radial_var_nonor_stack, axis=(0, 1))
print(radial_var_nonor_sum.shape)

fig, ax = plt.subplots(1, 2, figsize=(6, 3))
ax[0].plot(radial_var_sum)
ax[1].plot(radial_var_nonor_sum)
fig.tight_layout()
plt.show()

In [ ]:
# save (radial variance, 3D)
radial_var_stack_save = hys.signals.Signal1D(radial_var_stack)
radial_var_stack_save.metadata.General.set_item("notes", center_mean.tolist())
radial_var_stack_save.save(raw_adr[:-4]+"_radial_var_radial.hdf5")

In [ ]:
# save (radial variance, 3D)
radial_var_nonor_stack_save = hys.signals.Signal1D(radial_var_nonor_stack)
radial_var_nonor_stack_save.metadata.General.set_item("notes", center_mean.tolist())
radial_var_nonor_stack_save.save(raw_adr[:-4]+"_radial_var_nonor_radial.hdf5")

In [ ]:
radial_var = radial_var_nonor_stack.copy()
radial_var_spectrum = radial_var_nonor_sum.copy()

In [ ]:
radial_var = radial_var_stack.copy()
radial_var_spectrum = radial_var_sum.copy()

In [ ]:
def radial_indices(shape, radial_range, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    r = np.hypot(y - center[0], x - center[1])
    ri = np.ones(r.shape)
    
    if len(np.unique(radial_range)) > 1:
        ri[np.where(r <= radial_range[0])] = 0
        ri[np.where(r > radial_range[1])] = 0
        
    else:
        r = np.round(r)
        ri[np.where(r != round(radial_range[0]))] = 0
    
    return ri

In [ ]:
def round_step(arr, step_size=1):
    
    divide = np.around(arr / step_size)
    rounded = divide * step_size
    
    return rounded

In [ ]:
def point_circle(image, radius, center=None):
    
    y, x = np.indices(image.shape)
    y = y.astype("float64")
    x = x.astype("float64")
    
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    y -= center[0]
    x -= center[1]
    
    angle_arr = np.zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            angle_arr[i, j] = np.angle(complex(x[i, j], y[i, j]), deg=True)
            
    angle_arr = angle_arr + 180
    #print(angle_arr)
    r = np.hypot(y, x)
    r = np.around(r)
    ri = np.where(r == radius)
    angle_sel = angle_arr[ri]
    value_sel = image[ri]

    return angle_arr, angle_sel, value_sel

In [ ]:
# angle unit [degree]
def angular_correlation(angles, values):
    angle_diff = []
    corr_values = []
    angle_diff_candidates = np.arange(0, 361, 1)
    corr_val_total = np.zeros(angle_diff_candidates.shape)
    
    int_sqavg = np.square(np.mean(values))
    
    for i in range(len(angles)):
        temp_diff = np.abs(angles[i:] - angles[i])
        temp_diff = round_step(temp_diff)
        corr_temp = values[i:] * values[i]

        angle_diff.extend(temp_diff.tolist())
        corr_values.extend(corr_temp.tolist())
        
    angle_diff = np.asarray(angle_diff)
    corr_values = np.asarray(corr_values)
    
    uni_angle = np.unique(angle_diff)
    #print(uni_angle)
    
    for ang in uni_angle:
        temp_ind = np.where(angle_diff==ang)
        corr_val_total[int(ang)] = np.mean(corr_values[temp_ind]) - int_sqavg

    return (corr_val_total / int_sqavg), corr_val_total

# rotational symmetry evaluation directly from DPs

In [ ]:
k_range = np.arange(25, 30, 1)

In [ ]:
start_time = time.process_time()
flatten_k = []
for k in k_range:
    temp = []
    for i in range(cr_shape[0]):
        for j in range(cr_shape[1]):
            angle_map, angle_sel, value_sel = point_circle(stack_4d_cropped[i, j], k, center=center_mean.tolist())
            angle_range = np.arange(0, 361, 1)
            flat_val = np.zeros(angle_range.shape)
            uni_angle = np.unique(angle_sel)
            for ang in uni_angle:
                temp_ind = np.where(angle_sel==ang)
                flat_val[int(ang)] = value_sel[temp_ind]
            temp.append(flat_val)     
    flatten_k.append(temp)
    print("%d radius completed"%(k))
    print("%d seconds have passed"%(time.process_time()-start_time))
print("all done")

In [ ]:
flatten_k = np.asarray(flatten_k).reshape(len(k_range), cr_shape[0], cr_shape[1], -1)
print(flatten_k.shape)

In [ ]:
%matplotlib qt

In [ ]:
%matplotlib inline

In [ ]:
flatk_fft = np.abs(np.fft.fft(flatten_k))
print(flatk_fft.shape)
flatk_fft_sum = np.sum(flatk_fft, axis=(1, 2))
print(flatk_fft_sum.shape)

In [ ]:
flatk_ps = periodogram(flatten_k)[1]
print(flatk_ps.shape)
flatk_ps_sum = np.sum(flatk_ps, axis=(1, 2))
print(flatk_ps_sum.shape)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(5, 10))
ax1 = ax[0].imshow(np.flip(flatk_fft_sum[:, 1:11].T, 0), cmap="afmhot", 
                   extent=[k_range[0], k_range[-1], 0.5, 10.5], aspect=5, 
                   vmax=np.percentile(flatk_fft_sum, 95))
ax2 = ax[1].imshow(np.flip(flatk_ps_sum[:, 1:11].T, 0), cmap="afmhot", 
                   extent=[k_range[0], k_range[-1], 0.5, 10.5], aspect=5, 
                   vmax=np.percentile(flatk_ps_sum, 95))
fig.tight_layout()
plt.show()

In [ ]:
k_selected = 27
k_ind = np.where(k_range==k_selected)[0][0]
print(k_ind)
rot_sym = [2, 3, 4, 5, 6, 10]

flatk_rot = []
for r in rot_sym:
    flatk_rot.append(flatk_fft[k_ind, :, :, r]/np.max(flatk_fft[k_ind, :, :, r]))
flatk_rot = np.asarray(flatk_rot)
print(flatk_rot.shape)

In [ ]:
k_var_map = radial_var[:, :, k_selected]
for i, r in enumerate(rot_sym):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(flatk_rot[i], cmap="afmhot")
    ax[0].axis("off")
    ax[0].set_title("rotation symmetry %d"%r)
    ax[1].imshow(k_var_map, cmap="afmhot")
    ax[1].axis("off")
    ax[2].imshow(flatk_rot[i], cmap="afmhot", alpha=0.8)
    ax[2].contour(k_var_map, colors="k", alpha=1.0, levels=5)
    ax[2].axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
masked = radial_var[:, :, k_selected:(k_selected+1)]
masked = masked / np.max(masked)

p1_list = np.arange(50, 96, 1)
p2 = 99.00
v2 = np.percentile(masked, p2)
xcorr_val = []
mse_val = []
ssim_val = []

for i, r in enumerate(rot_sym):
    tmp_xcorr = []
    tmp_mse = []
    tmp_ssim = []
    for p1 in p1_list:
        v1 = np.percentile(masked, p1)
        bool_img = masked.copy()
        bool_img[np.where(masked < v1)] = 0
        bool_img[np.where(masked > v2)] = 0
        cluster_img = np.sum(bool_img, axis=2)
        cluster_img = cluster_img / np.max(cluster_img)

        tmp_xcorr.append(np.correlate(flatk_rot[i].flatten(), cluster_img.flatten()))
        tmp_mse.append(mean_squared_error(flatk_rot[i], cluster_img))
        tmp_ssim.append(ssim(flatk_rot[i], cluster_img))
        """
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        ax.imshow(ang_corr_k_sum, cmap="afmhot", alpha=0.8)
        ax.axis("off")
        ax.contour(cluster_img, colors="k", alpha=1.0, levels=1)
        ax.set_title("%.2f%% percent to 99.99%%\nxcorr=%d, mse=%.4f, ssim=%.4f"%(p1, xcorr_val[-1], mse_val[-1], ssim_val[-1]))
        ax.axis("off")    
        fig.tight_layout()
        plt.show()
        """
    xcorr_val.append(tmp_xcorr)
    mse_val.append(tmp_mse)
    ssim_val.append(tmp_ssim)

In [ ]:
for i, r in enumerate(rot_sym):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].plot(p1_list, xcorr_val[i], "k-")
    ax[0].grid()
    ax[1].plot(p1_list, mse_val[i], "k-")
    ax[1].grid()
    ax[1].set_title("rotation symmetry %d"%r)
    ax[2].plot(p1_list, ssim_val[i], "k-")
    ax[2].grid()
    plt.show()

In [ ]:
flatk_sum_rot = np.sum(flatk_fft[k_ind, :, :, rot_sym], axis=0)
flatk_sum_rot = flatk_sum_rot / np.max(flatk_sum_rot)
print(flatk_sum_rot.shape)

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(flatk_sum_rot, cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(k_var_map, cmap="afmhot")
ax[1].axis("off")
ax[2].imshow(flatk_sum_rot, cmap="afmhot", alpha=0.8)
ax[2].contour(k_var_map, colors="k", alpha=1.0, levels=5)
ax[2].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
masked = radial_var[:, :, k_selected:(k_selected+1)]
masked = masked / np.max(masked)

p1_list = np.arange(50, 96, 1)
p2 = 99.00
v2 = np.percentile(masked, p2)
xcorr_val = []
mse_val = []
ssim_val = []

for p1 in p1_list:
    v1 = np.percentile(masked, p1)
    bool_img = masked.copy()
    bool_img[np.where(masked < v1)] = 0
    bool_img[np.where(masked > v2)] = 0
    cluster_img = np.sum(bool_img, axis=2)
    cluster_img = cluster_img / np.max(cluster_img)
    
    xcorr_val.append(np.correlate(flatk_sum_rot.flatten(), cluster_img.flatten()))
    mse_val.append(mean_squared_error(flatk_sum_rot, cluster_img))
    ssim_val.append(ssim(flatk_sum_rot, cluster_img))
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.imshow(flatk_sum_rot, cmap="afmhot", alpha=0.8)
    ax.axis("off")
    ax.contour(cluster_img, colors="k", alpha=1.0, levels=5)
    ax.set_title("%.2f%% percent to 99.99%%\nxcorr=%d, mse=%.4f, ssim=%.4f"%(p1, xcorr_val[-1], mse_val[-1], ssim_val[-1]))
    ax.axis("off")    
    fig.tight_layout()
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(p1_list, xcorr_val, "k-")
ax[0].grid()
ax[1].plot(p1_list, mse_val, "k-")
ax[1].grid()
ax[2].plot(p1_list, ssim_val, "k-")
ax[2].grid()
plt.show()

# angular correlation

In [ ]:
%matplotlib inline

In [ ]:
start_time = time.process_time()
angular_corr_nor = []
angular_corr_nonor = []
print(k_range.shape)
for k in k_range:
    temp = []
    temp_nonor = []
    for i in range(cr_shape[0]):
        for j in range(cr_shape[1]):
            angle_map, angle_sel, value_sel = point_circle(stack_4d_cropped[i, j], k, center=center_mean.tolist())
            ang_cor, ang_cor_nonor = angular_correlation(angle_sel, value_sel)
            temp.append(ang_cor)
            temp_nonor.append(ang_cor_nonor)
    angular_corr_nor.append(temp)
    angular_corr_nonor.append(temp_nonor)
    print("%d radius completed"%(k))
    print("%d seconds have passed"%(time.process_time()-start_time))
print("all done")

In [ ]:
angular_corr = angular_corr_nor

In [ ]:
angular_corr = np.asarray(angular_corr).reshape(len(k_range), cr_shape[0], cr_shape[1], -1)
print(angular_corr.shape)
ang_corr_sum = np.mean(angular_corr, axis=(1, 2))
print(ang_corr_sum.shape)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(angle_map, cmap="gray")
ax[0].axis("off")
ax[1].imshow(np.flip(ang_corr_sum.T, 0), cmap="afmhot", extent=[k_range[0], k_range[-1], angle_range[0]/10, angle_range[-1]/10])
fig.tight_layout()
plt.show()

In [ ]:
ang_corr_fft = np.abs(np.fft.fft(angular_corr))
print(ang_corr_fft.shape)
ang_corr_fft_sum = np.abs(np.fft.fft(ang_corr_sum))
print(ang_corr_fft_sum.shape)

In [ ]:
ang_ps = periodogram(angular_corr)[1]
print(ang_ps.shape)
ang_ps_sum = periodogram(ang_corr_sum)[1]
print(ang_ps_sum.shape)

In [ ]:
%matplotlib qt

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(5, 10))
ax1 = ax[0].imshow(np.flip(ang_corr_fft_sum[:, 1:11].T, 0), cmap="afmhot", 
                   extent=[k_range[0], k_range[-1], 0.5, 10.5], aspect=5, 
                   vmax=np.max(ang_corr_fft_sum))
ax2 = ax[1].imshow(np.flip(ang_ps_sum[:, 1:11].T, 0), cmap="afmhot", 
                   extent=[k_range[0], k_range[-1], 0.5, 10.5], aspect=5, 
                   vmax=np.max(ang_ps_sum))
fig.tight_layout()
plt.show()

In [ ]:
k_selected = 25
k_ind = np.where(k_range==k_selected)[0][0]
print(k_ind)
rot_sym = [2, 3, 4, 5, 6, 10]

ang_corr_rot = []
for r in rot_sym:
    ang_corr_rot.append(ang_corr_fft[k_ind, :, :, r]/np.max(ang_corr_fft[k_ind, :, :, r]))
    #ang_corr_rot.append(ang_ps[k_ind, :, :, r]/np.max(ang_ps[k_ind, :, :, r]))
ang_corr_rot = np.asarray(ang_corr_rot)
print(ang_corr_rot.shape)

In [ ]:
k_var_map = radial_var[:, :, k_selected]
for i, r in enumerate(rot_sym):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(ang_corr_rot[i], cmap="afmhot")
    ax[0].axis("off")
    ax[0].set_title("rotation symmetry %d"%r)
    ax[1].imshow(k_var_map, cmap="afmhot")
    ax[1].axis("off")
    ax[2].imshow(ang_corr_rot[i], cmap="afmhot", alpha=0.8)
    ax[2].contour(k_var_map, colors="k", alpha=1.0, levels=5)
    ax[2].axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
masked = radial_var[:, :, k_selected:(k_selected+1)]
masked = masked / np.max(masked)

p1_list = np.arange(50, 96, 1)
p2 = 99.00
v2 = np.percentile(masked, p2)
xcorr_val = []
mse_val = []
ssim_val = []

for i, r in enumerate(rot_sym):
    tmp_xcorr = []
    tmp_mse = []
    tmp_ssim = []
    for p1 in p1_list:
        v1 = np.percentile(masked, p1)
        bool_img = masked.copy()
        bool_img[np.where(masked < v1)] = 0
        bool_img[np.where(masked > v2)] = 0
        cluster_img = np.sum(bool_img, axis=2)
        cluster_img = cluster_img / np.max(cluster_img)

        tmp_xcorr.append(np.correlate(ang_corr_rot[i].flatten(), cluster_img.flatten()))
        tmp_mse.append(mean_squared_error(ang_corr_rot[i], cluster_img))
        tmp_ssim.append(ssim(ang_corr_rot[i], cluster_img))
        """
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        ax.imshow(ang_corr_k_sum, cmap="afmhot", alpha=0.8)
        ax.axis("off")
        ax.contour(cluster_img, colors="k", alpha=1.0, levels=1)
        ax.set_title("%.2f%% percent to 99.99%%\nxcorr=%d, mse=%.4f, ssim=%.4f"%(p1, xcorr_val[-1], mse_val[-1], ssim_val[-1]))
        ax.axis("off")    
        fig.tight_layout()
        plt.show()
        """
    xcorr_val.append(tmp_xcorr)
    mse_val.append(tmp_mse)
    ssim_val.append(tmp_ssim)

In [ ]:
for i, r in enumerate(rot_sym):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].plot(p1_list, xcorr_val[i], "k-")
    ax[0].grid()
    ax[1].plot(p1_list, mse_val[i], "k-")
    ax[1].grid()
    ax[1].set_title("rotation symmetry %d"%r)
    ax[2].plot(p1_list, ssim_val[i], "k-")
    ax[2].grid()
    plt.show()

In [ ]:
masked = radial_var[:, :, k_selected:(k_selected+1)]

p1_list = np.arange(50, 96, 1)
p2 = 99.00
v2 = np.percentile(masked, p2)
xcorr_val = []
mse_val = []
ssim_val = []

for i, r in enumerate(rot_sym):
    tmp_xcorr = []
    tmp_mse = []
    tmp_ssim = []
    for p1 in p1_list:
        v1 = np.percentile(masked, p1)
        bool_img = np.ones_like(masked)
        bool_img[np.where(masked < v1)] = 0
        bool_img[np.where(masked > v2)] = 0
        bool_img = np.sum(bool_img, axis=2)

        db = DBSCAN(eps=1.5, min_samples=4)
        sel_coor = np.nonzero(bool_img)
        X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
        db.fit(X)
        label = db.labels_
        img_tmp = np.zeros_like(bool_img)
        img_tmp += -1
        img_tmp[X[:, 0], X[:, 1]] = label
        cluster_img = np.where(img_tmp<0, img_tmp, 1)
        cluster_img = np.where(cluster_img>-1, bool_img, 0)

        tmp_xcorr.append(np.correlate(ang_corr_rot[i].flatten(), cluster_img.flatten()))
        tmp_mse.append(mean_squared_error(ang_corr_rot[i], cluster_img))
        tmp_ssim.append(ssim(ang_corr_rot[i], cluster_img))
        """
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        ax.imshow(ang_corr_k_sum, cmap="afmhot", alpha=0.8)
        ax.axis("off")
        ax.contour(cluster_img, colors="k", alpha=1.0, levels=1)
        ax.set_title("%.2f%% percent to 99.99%%\nxcorr=%d, mse=%.4f, ssim=%.4f"%(p1, xcorr_val[-1], mse_val[-1], ssim_val[-1]))
        ax.axis("off")    
        fig.tight_layout()
        plt.show()
        """
    xcorr_val.append(tmp_xcorr)
    mse_val.append(tmp_mse)
    ssim_val.append(tmp_ssim)

In [ ]:
for i, r in enumerate(rot_sym):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].plot(p1_list, xcorr_val[i], "k-")
    ax[0].grid()
    ax[1].plot(p1_list, mse_val[i], "k-")
    ax[1].grid()
    ax[1].set_title("rotation symmetry %d"%r)
    ax[2].plot(p1_list, ssim_val[i], "k-")
    ax[2].grid()
    plt.show()

In [ ]:
k_selected = 25
k_ind = np.where(k_range==k_selected)[0][0]
print(k_ind)
rot_sym = [2, 3, 4, 5, 6, 10]
ang_corr_k_sum = np.sum(ang_corr_fft[k_ind, :, :, rot_sym], axis=0)
#ang_corr_k_sum = np.sum(ang_ps[k_ind, :, :, rot_sym], axis=0)
ang_corr_k_sum = ang_corr_k_sum / np.max(ang_corr_k_sum)
print(ang_corr_k_sum.shape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.imshow(ang_corr_k_sum, cmap="afmhot")
ax.axis("off")
fig.tight_layout()
plt.show()

In [ ]:
masked = radial_var[:, :, k_selected:(k_selected+1)]
masked = masked / np.max(masked)

p1_list = np.arange(50, 96, 1)
p2 = 99.00
v2 = np.percentile(masked, p2)
xcorr_val = []
mse_val = []
ssim_val = []

for p1 in p1_list:
    v1 = np.percentile(masked, p1)
    bool_img = masked.copy()
    bool_img[np.where(masked < v1)] = 0
    bool_img[np.where(masked > v2)] = 0
    cluster_img = np.sum(bool_img, axis=2)
    cluster_img = cluster_img / np.max(cluster_img)
    
    xcorr_val.append(np.correlate(ang_corr_k_sum.flatten(), cluster_img.flatten()))
    mse_val.append(mean_squared_error(ang_corr_k_sum, cluster_img))
    ssim_val.append(ssim(ang_corr_k_sum, cluster_img))
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.imshow(ang_corr_k_sum, cmap="afmhot", alpha=0.8)
    ax.axis("off")
    ax.contour(cluster_img, colors="k", alpha=1.0, levels=5)
    ax.set_title("%.2f%% percent to 99.99%%\nxcorr=%d, mse=%.4f, ssim=%.4f"%(p1, xcorr_val[-1], mse_val[-1], ssim_val[-1]))
    ax.axis("off")    
    fig.tight_layout()
    plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(5, 8))
ax[0].plot(p1_list, xcorr_val, "k-")
ax[0].grid()
ax[1].plot(p1_list, mse_val, "k-")
ax[1].grid()
ax[2].plot(p1_list, ssim_val, "k-")
ax[2].grid()
plt.show()

In [ ]:
masked = radial_var[:, :, k_selected:(k_selected+1)]

p1_list = np.arange(50, 96, 1)
p2 = 99.00
v2 = np.percentile(masked, p2)
xcorr_val = []
mse_val = []
ssim_val = []

for p1 in p1_list:
    v1 = np.percentile(masked, p1)
    bool_img = np.ones_like(masked)
    bool_img[np.where(masked < v1)] = 0
    bool_img[np.where(masked > v2)] = 0
    bool_img = np.sum(bool_img, axis=2)
    
    db = DBSCAN(eps=1.5, min_samples=4)
    sel_coor = np.nonzero(bool_img)
    X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
    db.fit(X)
    label = db.labels_
    img_tmp = np.zeros_like(bool_img)
    img_tmp += -1
    img_tmp[X[:, 0], X[:, 1]] = label
    cluster_img = np.where(img_tmp<0, img_tmp, 1)
    cluster_img = np.where(cluster_img>-1, bool_img, 0)
    
    xcorr_val.append(np.correlate(ang_corr_k_sum.flatten(), cluster_img.flatten()))
    mse_val.append(mean_squared_error(ang_corr_k_sum, cluster_img))
    ssim_val.append(ssim(ang_corr_k_sum, cluster_img))
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.imshow(ang_corr_k_sum, cmap="afmhot", alpha=0.8)
    ax.axis("off")
    ax.contour(cluster_img, colors="k", alpha=1.0, levels=5)
    ax.set_title("%.2f%% percent to 99.99%%\nxcorr=%d, mse=%.4f, ssim=%.4f"%(p1, xcorr_val[-1], mse_val[-1], ssim_val[-1]))
    ax.axis("off")    
    fig.tight_layout()
    plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(5, 8))
ax[0].plot(p1_list, xcorr_val, "k-")
ax[0].grid()
ax[1].plot(p1_list, mse_val, "k-")
ax[1].grid()
ax[2].plot(p1_list, ssim_val, "k-")
ax[2].grid()
plt.show()

# large variance & STEM

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(15, 5))
G = gridspec.GridSpec(2, 5)
ax0 = plt.subplot(G[0, 0])
ax1 = plt.subplot(G[0, 1])
ax2 = plt.subplot(G[0, 2])
ax3 = plt.subplot(G[1, 0])
ax4 = plt.subplot(G[1, 1])
ax5 = plt.subplot(G[1, 2])
ax6 = plt.subplot(G[:, 3:])

radii = np.arange(len(radial_var_sum))
#pacbed = np.mean(stack_4d_cropped, axis=(0, 1))
pacbed = np.mean(stack_4d_cropped, axis=(0, 1))
cluster_result = ["dummy"]

def var_selector(di, do, uth, lth, k1, k2, p1, p2):
    
    v1 = np.percentile(radial_var[:, :, k1:(k2+1)], p1)
    v2 = np.percentile(radial_var[:, :, k1:(k2+1)], p2)
    
    ax0.cla()
    ax1.cla()
    ax2.cla()
    ax3.cla()
    ax4.cla()
    ax5.cla()
    ax6.cla()

    ax0.plot(radii, radial_var_sum, "k-", c="black")
    ax0.fill_between([k1, k2], np.max(radial_var_spectrum), alpha=0.5, color="orange")
    ax0.set_title("rotational variance profile")
    ax0.grid()
    
    del cluster_result[:]
    
    masked = radial_var[:, :, k1:(k2+1)]
    binsize = masked.shape[0]*masked.shape[1]
    ax1.hist(masked.flatten(), bins=binsize, color="darkgray")
    ax1.fill_between([v1, v2], ax1.get_ylim()[1], alpha=0.5, color="orange")
    ax1.set_title("variance histogram for a selected k-range")
    ax1.grid()
    
    bool_img = np.ones_like(masked)
    bool_img[np.where(masked < v1)] = 0
    bool_img[np.where(masked > v2)] = 0
    img = np.sum(bool_img, axis=2)
    ax2.imshow(img, cmap="afmhot")
    ax2.set_title("variance map, larger than threshold")
    ax2.axis("off")
    
    cluster_result.append(img)
    
    if di > do:
        det = [do, di]
        print("Warning! detector 1 (inner angle > outer angle)")
    else:
        det = [di, do]
        
        
    ri = radial_indices(f_shape[2:], det, center=center_mean.tolist())
    
    ax3.imshow(pacbed, cmap="gray")
    ax3.imshow(ri, cmap="Reds", alpha=0.3)
    ax3.set_title("detector range")
    ax3.axis("off")
    
    stem = np.sum(np.multiply(stack_4d_cropped, ri), axis=(2, 3))
    ax4.imshow(stem, cmap="afmhot")
    ax4.set_title("STEM image")
    ax4.axis("off")
    
    stem_bool = np.ones_like(stem)
    stem_bool[np.where(stem > np.percentile(stem, uth))] = 0.0
    stem_bool[np.where(stem <= np.percentile(stem, lth))] = 0.0
    
    ax5.imshow(stem_bool, cmap="summer")
    ax5.set_title("STEM, larger than threshold")
    ax5.axis("off")

    ax6.imshow(stem_bool, cmap="summer")
    ax6.imshow(img, cmap="afmhot", alpha=0.3)
    ax6.set_title("variance vs. STEM")
    ax6.axis("off")
    
    fig.canvas.draw()
    fig.tight_layout()

st = {"description_width": "initial"}
din = pyw.IntText(value=25, description="D1 inner angle: ", style=st)
dout = pyw.IntText(value=30, description="D1 outer angle: ", style=st)
up_th = pyw.IntText(value=10, description="STEM upper threshold: ", style=st)
lo_th = pyw.IntText(value=0, description="STEM lower threshold: ", style=st)
k1_widget = pyw.IntText(value=25, description="left of k-range", style=st)
k2_widget = pyw.IntText(value=30, description="right of k-range", style=st)
p1_widget = pyw.FloatText(value=90.0, description="left percentile of variance range", style=st)
p2_widget = pyw.FloatText(value=99.999, description="right percentile of variance range", style=st)

pyw.interact(var_selector, di=din, do=dout, uth=up_th, lth=lo_th, k1=k1_widget, k2=k2_widget, p1=p1_widget, p2=p2_widget)
fig.show()